In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import glob
import os
import warnings

from astropy.io import ascii
from astropy.io import fits

%matplotlib inline

In [2]:
igrins_log = pd.read_csv('./xmatch_log.csv') # RRSIA reduced data log
igrins_log = igrins_log.drop(labels=['FILENUMBER', 'FILES', 'STANDARD', 'JD', 'OBJTYPE', 'EXPTIME',
       'ROTPA', 'AM', 'BVC', 'FACILITY', 'PI', 'PROGID', 'RA_s', 'DEC_s',
       'IDS', 'OTYPE', 'SP_TYPE', 'SP_BIBCODE', 'PMRA', 'PMDEC', 'PM_BIBCODE',
       'RV_VALUE', 'RV_BIBCODE', 'PLX_VALUE', 'PLX_BIBCODE',
       'J', 'H', 'K', '2MASS_ID', '2MASS_J', '2MASS_H',
       '2MASS_K', '2MASS_Flag', 'GaiaDR3_source', 'GaiaDR3_parallax',
       'GaiaDR3_pm', 'GaiaDR3_bprp', 'GaiaDR3_ebprp', 'GaiaDR3_gmag',
       'GaiaDR3_RV', 'GaiaDR3_teff', 'GaiaDR3_logg', 'GaiaDR3_FeH',
       'GaiaDR3_dist', 'GaiaEDR3_Flag', 'APOGEE2_HRV', 'APOGEE2_teff',
       'APOGEE2_logg', 'APOGEE2_Vsini', 'APOGEE2_[M/H]', 'APOGEE2_[a/M]', 'APOGEE2_[Fe/H]'],axis=1) # drop a few columns

luhman_cat = pd.read_table('./apjs328021t7_sptype_ascii.txt',header=0) # luhman data table
luhman_sptypes = luhman_cat['SED_Class'] # SED class column

class_3_mask = luhman_sptypes == 'III' # conditional statement
luhman_class3 = luhman_cat[class_3_mask] # apply mask
luhman_class3.head() # all class 3 objects in Luhman's catalog (2008 I think?)

# unique targets of observation from RRISA
unique_rrisa = pd.Series(igrins_log['OBJNAME_super'].unique())

# Filter and sort the DataFrame based on match_mask
match_mask = luhman_class3['Name'].isin(unique_rrisa)  # check the names from RRISA against Luhman class 3 objects
luhman_class3 = luhman_class3[match_mask].sort_values(['Spectral_Type'])  # sort by Spectral_Type

# Reset the index
luhman_class3 = luhman_class3.reset_index(drop=True)  # reset index to start from 0 and drop the old index

# Extract the names of the class 3 objects that are also in RRISA
class3_objects = luhman_class3['Name']

# these are the class 3 standards that should have IGRINS data in RRISA after dropping some columns
luhman_class3 = luhman_class3.drop(columns=["alpha(2-8 mum)", "alpha(2-24 mum)", "alpha(3.6-8 mum)",	"alpha(3.6-24 mum)", "Unnamed: 8"])

In [3]:
igrins_log.head()

,NAME,OBJNAME_super,OBJNAME_recipe,MAIN_ID,CIVIL,RA,DEC,FILENAME,SNRH_pix,SNRH_res,...,R,G,I,PASTEL_Teff,PASTEL_logg,PASTEL_[Fe/H],PASTEL_Flag,FILE_URL,CAL_URL,RAW_URL
0,* mu.02 Her,LHS 3325,LHS 3325,* mu.02 Her,20140707,266.771375,27.727028,SDCH_20140707_0095.fits,231.749786,446,...,9.400,8.9773,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/8roandzn8...,https://utexas.box.com/shared/static/hodoh169z...,https://utexas.box.com/s/p5esie4bqbecxlnv7tm6k...
1,* alf Lyr,vega,vega,* alf Lyr,20140707,279.393667,38.804667,SDCH_20140707_0099.fits,551.093567,1059,...,0.070,NaN,0.1,0.0,NaN,-0.30,NaN,https://utexas.box.com/shared/static/eb5fsr3ci...,https://utexas.box.com/shared/static/hodoh169z...,https://utexas.box.com/s/p5esie4bqbecxlnv7tm6k...
2,HD 164595B,NLTT45791,NLTT45791,HD 164595B,20140707,270.353292,29.581639,SDCH_20140707_0111.fits,60.458027,116,...,12.173,11.6237,NaN,0.0,NaN,-0.07,NaN,https://utexas.box.com/shared/static/mccb93th9...,https://utexas.box.com/shared/static/hodoh169z...,https://utexas.box.com/s/p5esie4bqbecxlnv7tm6k...
3,NAME BD+31 3330BC,NLTT46858,NLTT46858,NAME BD+31 3330BC,20140707,280.389208,31.558806,SDCH_20140707_0116.fits,124.143341,239,...,10.839,10.6177,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/ysd40crbh...,https://utexas.box.com/shared/static/hodoh169z...,https://utexas.box.com/s/p5esie4bqbecxlnv7tm6k...
4,L 1288-4,GJ797B,GJ797B,L 1288-4,20140707,310.365000,19.958444,SDCH_20140707_0124.fits,106.686111,205,...,11.458,11.3935,NaN,0.0,NaN,-0.07,NaN,https://utexas.box.com/shared/static/jk0ajudat...,https://utexas.box.com/shared/static/hodoh169z...,https://utexas.box.com/s/p5esie4bqbecxlnv7tm6k...


In [4]:
unique_rrisa

0                  LHS 3325
1                      vega
2                 NLTT45791
3                 NLTT46858
4                    GJ797B
               ...         
5911               HD118100
5912               HD116544
5913               TOI-1842
5914    4042279187272458496
5915         EPIC 224328215
Length: 5916, dtype: object

In [24]:
luhman_class3['Name']

0           HD 30171
1          HD 283572
2            LkCa 19
3            HBC 427
4     RXJ 05072+2437
5             LkCa 4
6           Hubble 4
7           V830 Tau
8           L1551-55
9           V827 Tau
10          L1551-51
11            Anon 1
12       XEST 09-042
13            JH 108
14           HBC 359
15          UX Tau B
16            LkCa 5
17           LkCa 21
18            J1-507
19            LkCa 1
20         Haro 6-32
21       XEST 20-066
22            J1-665
23             MHO 8
24             MHO 4
Name: Name, dtype: object

In [11]:
import requests
import ast

igrins_class3 = igrins_log[igrins_log['OBJNAME_super'].isin(luhman_class3['Name'])].reset_index(drop=True)
# RRISA table of Class 3 objects with Luhman spectral classifcations
igrins_class3

,NAME,OBJNAME_super,OBJNAME_recipe,MAIN_ID,CIVIL,RA,DEC,FILENAME,SNRH_pix,SNRH_res,...,R,G,I,PASTEL_Teff,PASTEL_logg,PASTEL_[Fe/H],PASTEL_Flag,FILE_URL,CAL_URL,RAW_URL
0,HD 282630,LkCa 19,LkCa 19,HD 282630,20141125,74.157167,30.321944,SDCH_20141125_0127.fits,49.880112,96,...,10.55,10.34040,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/ocd0q79da...,https://utexas.box.com/shared/static/zpg3qqqlc...,https://utexas.box.com/s/pmel1g5f3ij38rnu1yfty...
1,V* V830 Tau,V830 Tau,V830 Tau,V* V830 Tau,20141127,68.517708,24.583833,SDCH_20141127_0104.fits,59.686356,115,...,11.30,11.66720,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/y5vynuy46...,https://utexas.box.com/shared/static/wkwhqyltt...,https://utexas.box.com/s/4szm51p24fvn7suygk5ul...
2,V* V1095 Tau,LkCa 1,LkCa 1,V* V1095 Tau,20141127,63.545958,28.350444,SDCH_20141127_0112.fits,61.841385,119,...,13.62,12.51900,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/v24kuk9oo...,https://utexas.box.com/shared/static/wkwhqyltt...,https://utexas.box.com/s/4szm51p24fvn7suygk5ul...
3,L 1551-51,L1551-51,L1551-51,L 1551-51,20141129,68.256542,17.983639,SDCH_20141129_0129.fits,48.081284,93,...,NaN,11.64780,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/j9ie8gkof...,https://utexas.box.com/shared/static/gj7lu2x64...,https://utexas.box.com/s/io5r8rqtxffztqorjrh58...
4,HD 283572,HD 283572,HD 283572,HD 283572,20141130,65.723208,28.336472,SDCH_20141130_0145.fits,74.839287,144,...,9.14,8.80347,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/h22ss8iqg...,https://utexas.box.com/shared/static/nqlk0kd1d...,https://utexas.box.com/s/5pu2aajnfb04d2os5v94f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,V* V830 Tau,V830 Tau,V830 Tau,V* V830 Tau,20190309,68.291083,24.564361,SDCH_20190309_0035.fits,56.529030,109,...,11.30,11.66720,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/x1up2o88d...,https://utexas.box.com/shared/static/rfb7ul7qs...,https://utexas.box.com/s/t9qttei7ftpxtggkri8hn...
276,V* V827 Tau,V827 Tau,V827 Tau,V* V827 Tau,20190309,68.059958,18.339750,SDCH_20190309_0041.fits,74.186844,143,...,12.35,11.86990,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/qm5qzximp...,https://utexas.box.com/shared/static/rfb7ul7qs...,https://utexas.box.com/s/t9qttei7ftpxtggkri8hn...
277,V* V830 Tau,V830 Tau,V830 Tau,V* V830 Tau,20190314,68.290333,24.563528,SDCH_20190314_0039.fits,50.843269,98,...,11.30,11.66720,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/jme633dz5...,https://utexas.box.com/shared/static/4m3ruyifs...,https://utexas.box.com/s/kj5g275o29irzj8taaslb...
278,V* V830 Tau,V830 Tau,V830 Tau,V* V830 Tau,20190316,68.291250,24.563861,SDCH_20190316_0033.fits,68.080223,131,...,11.30,11.66720,NaN,NaN,NaN,NaN,NaN,https://utexas.box.com/shared/static/q7iam1pvf...,https://utexas.box.com/shared/static/stip8b0xf...,https://utexas.box.com/s/ypn0575pyyvzujhvfep4n...


In [79]:
file_path = 'C:/Users/Savio/Documents/IGRINS-Spectra/Standards/MHO_8/20190124/20190124_008/SDCK_20190124_0085.spec_a0v.fits'

standards_speca0v_list = glob.glob('C:/Users/Savio/Documents/IGRINS-Spectra/Standards/*/2018*/*/SDCK*.spec_a0v.fits')

test_hdu = fits.open(standards_speca0v_list[0])
test_hdu.info()

flux = test_hdu[1].data # SPEC_DIVIDE_A0V
wavelength = test_hdu[3].data # WAVELENGTH

# test_hdr = fits.getheader(standards_speca0v_list[0])
# test_data = fits.getdata(standards_speca0v_list[0])
# test_data.shape

Filename: C:/Users/Savio/Documents/IGRINS-Spectra/Standards\Anon_1\20181120\20181120_007\SDCK_20181120_0075.spec_a0v.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     100   ()      
  1  SPEC_DIVIDE_A0V    1 ImageHDU         8   (2048, 25)   float64   
  2  SPEC_DIVIDE_A0V_VARIANCE    1 ImageHDU         8   (2048, 25)   float64   
  3  WAVELENGTH    1 ImageHDU         8   (2048, 25)   float64   
  4  TGT_SPEC      1 ImageHDU       213   (2048, 25)   float32   
  5  TGT_SPEC_VARIANCE    1 ImageHDU         8   (2048, 25)   float32   
  6  A0V_SPEC      1 ImageHDU       213   (2048, 25)   float32   
  7  A0V_SPEC_VARIANCE    1 ImageHDU         8   (2048, 25)   float32   
  8  VEGA_SPEC     1 ImageHDU         8   (2048, 25)   float64   
  9  SPEC_DIVIDE_CONT    1 ImageHDU         8   (2048, 25)   float64   
 10  SPEC_DIVIDE_CONT_VARIANCE    1 ImageHDU         8   (2048, 25)   float64   
 11  MASK          1 ImageHDU         8   (2048, 25

In [64]:
test_hdu[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
I_HDRVER= '0.98    '           / version of IGRINS FITS Header                  
OBSERVAT= 'Lowell Observatory' / Name of the observatory                        
TELESCOP= 'Discovery Channel'  / Name of the telescope                          
INSTRUME= 'IGRINS  '           / Name of the instrument                         
DETECTOR= 'H2RG    '           / name of Detector                               
TIMESYS = 'UTC     '           / Time system used in this header                
OBSERVER= 'Karnath, Habel'     / Observer                                       
OBJECT  = 'Anon 1  '           / name of Object                                 
EXPTIME =                200

In [61]:
wavelength.shape

(25, 2048)

In [60]:
flux.shape

(25, 2048)